In [2]:
# Load data 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)
# Load MNIST dataset
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# split train data in two
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

Xtrain = Xtrain[0:30001, :, :];
Ytrain = Ytrain[0:30001]

# downsample
factor = 0.25

Xtrain_down = np.ones((Xtrain.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtrain.shape[0]):
    Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], factor)

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

    
X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# *** VECTORIZE IMAGES ***
Xtrain_down = Xtrain_down.reshape(Xtrain_down.shape[0], int(xdim*factor)**2).astype('float32') / 255
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

In [3]:
# Load NN and get output values
from keras import backend as K
from keras.models import load_model

model = load_model('nn.h5')

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function

# Testing: layer_outs is a list with 4 data sets
layer_outs = functor([X_pr_down, 1.])

layer_outs_test= functor([Xtest_down, 1.])



In [4]:
# subsample and do xgboost regression
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData

import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier

# setup parameters for xgboost
params = {}
# use softmax multi-class classification
params['objective'] = 'multi:softmax'
# scale weight of positive examples
params['eta'] = 0.1
params['max_depth'] = 6
params['silent'] = 1
params['nthread'] = 4
params['num_class'] = 10
num_round=5

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = range(49)+1
nSubnetSize = len(subnetSize)
# which layers?
# how many samples per recording?
nSamples = X_pr_down.shape[0]/nRecordings
# how many iterations
nIterations = 100

oLayer = len(layer_outs)-1
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for ss in range(nSubnetSize):
    nLayerNeurons = [subnetSize[ss], 0, 0, 10]
    for it in range(nIterations):
        #print(nLayerNeurons, iter)
        # copy data
        layer_outputs = deepcopy(layer_outs)
        # subsample
        X= RE_PartialRecData(layer_outputs, nLayerNeurons, nRecordings, nSamples)
        # prepare data for xgboost
        xg_train = xgb.DMatrix(X, label=Y_pr)
        xg_test  = xgb.DMatrix(layer_outs_test[0], label=Ytest)
        watchlist = [ (xg_train,'train'), (xg_test, 'test') ]
        for iN in range(nOutNeurons):
            print(iN)
            xg_train = xgb.DMatrix(X, label=layer_outs[3][:,iN])
            xg_test  = xgb.DMatrix(layer_outs_test[0], label=layer_outs_test[3][:,iN])
            watchlist = [ (xg_train,'train'), (xg_test, 'test') ]

            bst = xgb.train(params, xg_train, num_round, watchlist);
            # get prediction
            pred = bst.predict( xg_test );
            rmses[it,iN,ss]=np.sqrt(sum(np.square(pred[i] -layer_outs_test[3][:,1][i]) 
                                                    for i in range(len(layer_outs_test[3][:,1]))) / float(len(layer_outs_test[3][:,1])))
            print ('predicting, RMSE=%f' %rmses[it, iN,ss ] )
    


/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


UnboundLocalError: local variable 'X' referenced before assignment

In [8]:
subnetSize = range(49)+1

[0, 0, 0, 10]

In [5]:
X= RE_PartialRecData(layer_outputs, nLayerNeurons, nRecordings, nSamples)

UnboundLocalError: local variable 'X' referenced before assignment

In [ ]:
Y[1:10, :
 ]

In [ ]:
oLayer = len(layer_outs)-1
oLayer

